# Raw tss from direct bmw responses
The goal of this notebook is to demonstrate how to parse the raw tss from direct bmw responses.

## Setup




### Imports

In [ ]:
from core.pandas_utils import *
from core.singleton_s3_bucket import bucket

## Implementation

Let's first take a look at an example response.

In [ ]:
EXAMPLE_KEY = "response/BMW/WBY71AW000FM68170/2024-12-02.json"
response = bucket.read_json_file(EXAMPLE_KEY)
response

In [ ]:
raw_tss = (
    DF.from_dict(response["data"])
    .pivot(
        index="date_of_value",
        columns="key",
        values="value",
    )
)
raw_tss

In [ ]:
RESPONSE2_KEY = "response/BMW/WBY71AW000FM68170/2024-11-30.json"
response2 = bucket.read_json_file(RESPONSE2_KEY)
raw_tss2 = (
    DF.from_dict(response2["data"])
    .pivot(
        index="date_of_value",
        columns="key",
        values="value",
    )
)
raw_tss2

In [ ]:
cat_raw_tss = pd.merge(raw_tss, raw_tss2, how="outer", on="date_of_value")#pd.concat([raw_tss, raw_tss2])
cat_raw_tss


In [ ]:
display(raw_tss.index.has_duplicates)
display(raw_tss2.index.has_duplicates)
display(cat_raw_tss.index[cat_raw_tss.index.duplicated()])

In [ ]:
raw_tss_dates_counts = raw_tss.index.value_counts()
cat_dates_counts = cat_raw_tss.index.value_counts()
display(raw_tss_dates_counts)
display(cat_dates_counts)

In [ ]:
raw_tss_dates_counts.eq(cat_dates_counts.loc[raw_tss_dates_counts.index]).value_counts(normalize=True)

In [ ]:
units = (
    DF.from_dict(response["data"])
    .pivot(
        index="date_of_value",
        columns="key",
        values="unit",
    )
    .agg(Series.mode, axis="index")
    .squeeze()
    .fillna("string")
    .sort_values()
)
units